In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from typing import Union, List
from sklearn.preprocessing import StandardScaler, MinMaxScaler # type: ignore
from sklearn.decomposition import PCA                   # type: ignore

import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings(action='ignore')

In [7]:
class Preprocessor:
    def __init__(self):
        self.file_path = ""
        self.folder_path = ""
        self.df = pd.DataFrame()
        self.pca_df = pd.DataFrame()

    def __init__(self, data_file_path:str="", folder_path:str=""):
        self.file_path = data_file_path
        self.folder_path = folder_path
        self.df = pd.DataFrame()
        self.pca_df = pd.DataFrame()
        
    def load_origin_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)
        
    def drop_columns(self, drop_columns_file_path:str = "drop_columns.txt")->None:
        with open(drop_columns_file_path, mode='r') as f:
            drop_fields = f.readlines()
            drop_fields = [drop_field.strip('\n') for drop_field in drop_fields]
        self.df.drop(columns=drop_fields, inplace=True)
    
    def __preprocess_target_variable(self, target_variable:str="loan_status")->None:
        # loan_status가 "current", "issued", "policy" 인 행을 필터링하여 삭제
        modified_df = self.df[~self.df[target_variable].isin(['Current', 'Issued', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]
        # risk = 1, safe = 0 으로 처리
        modified_df.loc[modified_df['loan_status'].isin(['Fully Paid', 'In Grace Period']), 'loan_status'] = 0
        modified_df.loc[modified_df['loan_status'].isin(['Charged Off', 'Default', 'Late (16-30 days)', 'Late (31-120 days)']), 'loan_status'] = 1
        modified_df['loan_status'] = modified_df['loan_status'].astype('int')
        self.df = modified_df
        
    ## 5. 데이터 처리용 함수
    def __delete_suffix(self, term:str)->int:
        '''첫 단어만을 저장하는 함수'''
        term = term.strip().split()[0]
        return int(term)

    def __delete_suffix_percentage(self, term:str)->float:
        '''%를 자르는 함수'''
        term = term.strip('%')
        return float(term)
    
    def __fill_na_with_value(self, columns:List[str], filling_value:Union[str, int])->None:
        '''
        df: dataframe to fill NA
        column_name : column name to change NA values
        filling_value : value type or just value to fill column's NA
        '''
        for column_name in columns:
            if filling_value=="mode":
                mode_value = self.df[column_name].mode()[0]
            elif filling_value=="median":
                mode_value = self.df[column_name].median()
            else:
                mode_value = filling_value
            self.df[column_name].fillna(mode_value, inplace=True)
        
    def __preprocessing_na(self, is_train:bool)->None:
        '''
        결측치에 대해서 전처리
        1. 해당 데이터 삭제
        2. 값 대체(0, 중앙값, 최빈값)
        3. 값 대체 후, 새로운 컬럼(is_after_2012, is_after_2015) 추가
        '''
        ## 결측 처리
        # 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
        self.df.dropna(subset=['chargeoff_within_12_mths','collections_12_mths_ex_med','dti',
                                                'pub_rec_bankruptcies','revol_util','tax_liens'], inplace=True)
        # 결측치 값 대체
        if is_train:
            # A1. 최빈값 대체
            self.__fill_na_with_value(columns=['mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_inq', 'emp_length'], filling_value='mode')
            
            # A2. 중앙값 대체
            self.__fill_na_with_value(columns=['bc_open_to_buy'], filling_value='median')
        
        # B. 2015년 대체
        # is_after_2015 컬럼 생성. all_util 변수를 기준으로 사용
        self.df['is_after_2015'] = self.df['all_util'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # C. 2012년 대체
        # is_after_2012 컬럼 생성. pct_tl_nvr_dlq 변수를 기준으로 사용
        self.df['is_after_2012'] = self.df['pct_tl_nvr_dlq'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # D. 결측 0 대체
        if is_train:
            self.__fill_na_with_value(columns=['open_acc_6m','open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il',
                                           'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_cu_tl', 'mths_since_rcnt_il',
                                           'tot_cur_bal', 'total_rev_hi_lim', 'mo_sin_old_rev_tl_op',
                                           'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_bc_sats', 'num_bc_tl',
                                           'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_il_tl',
                                           'num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd',
                                           'num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','tot_hi_cred_lim','total_bal_ex_mort',
                                           'total_bc_limit','total_il_high_credit_limit'], filling_value=0)
        self.__fill_na_with_value(columns=['annual_inc_joint','dti_joint', 'revol_bal_joint'], filling_value=0)
        
        
    def __convert_object_to_numeric(self, column_name:str)->pd.DataFrame:
        '''
        연속형 변수(object type)를 수치형 변수로 처리
        1. application_type
        2. sub_grade
        '''
        unique_values = sorted(self.df[column_name].unique())
        value_map = {value:index for index, value in enumerate(unique_values)}
        self.df[column_name] = self.df[column_name].apply(lambda x:value_map.get(x))
        return self.df
    
    def __convert_object_to_one_hot(self, column_name:str)->None:
        '''
        범주형 변수를 one-hot encoding
        '''
        encoded = pd.get_dummies(self.df[column_name], drop_first=True, prefix='cat')
        self.df = pd.concat([self.df, encoded], axis=1)
        self.df.drop(column_name, axis=1, inplace=True)
        
    def __preprocessing_objects(self)->None:
        '''
        object type의 변수 전처리
        1. suffix 처리
        2. numeric 변수로 변환
        3. 범주형 변수 one-hot encoding
        '''
        # term
        self.df['term'] = self.df['term'].apply(self.__delete_suffix)
        # emp_length
        self.df = self.df.dropna(subset=['emp_length'])
        self.df['emp_length'] = self.df['emp_length'].astype(str).apply(lambda x: x.replace(' years','').replace(' year','').replace('+','').replace('< 1', '0'))
        self.df['emp_length'] = self.df['emp_length'].astype(int)
        # revol_util
        self.df['revol_util'] = self.df['revol_util'].apply(self.__delete_suffix_percentage)
        self.df['int_rate'] = self.df['int_rate'].apply(self.__delete_suffix_percentage)
        ## numeric
        # application_type
        self.df = self.__convert_object_to_numeric('application_type')
        # sub_grade
        self.df = self.__convert_object_to_numeric('sub_grade')

        ## one-hot
        # home_ownership
        self.df['home_ownership'] = self.df['home_ownership'].replace(['ANY', 'OTHER', 'NONE'], 'OTHERS')
        self.__convert_object_to_one_hot('home_ownership')
        # self.__purpose()
        # purpose
        self.__convert_object_to_one_hot('purpose')
        self.__verification_status()

    def __Multicollinearity(self)->None:
        self.df['fico_avg'] = (self.df['fico_range_low'] + self.df['fico_range_high'])/2
        self.df.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)

    def __purpose(self):
        '''
        대출 목적(purpose) 전처리
        대출 목적이 필수적인가, 선택적인가에 대해서 파악 후, 전처리
        1. 필수적인 목적 : essential_purpose
            small_business, house, moving, medical, other
        2. 선택적인 목적 : optional_purpose
            wedding, vacation, car, home_improvement, educational
        3. 해당 값이 많은 경우 mapping을 하지 않음
            deb_consolidation, credit_card  
        '''
        self.df['purpose'] = self.df['purpose'].map({"small_business":"essential_purpose", 
                                                     "house":"essential_purpose", 
                                                     "moving":"essential_purpose",
                                                     "medical":"essential_purpose",
                                                     "renewable_energy":"essential_purpose",
                                                     "other":"essential_purpose",
                                                     "wedding":"optional_purpose",
                                                     "vacation":"optional_purpose",
                                                     "car":"optional_purpose",
                                                     "home_improvement":"optional_purpose",
                                                     "educational":"optional_purpose",
                                                     "debt_consolidation":"debt_consolidation",
                                                     "credit_card":"credit_card",})
    def __verification_status(self):
        self.df['verification_status'] = self.df['verification_status'].map({"Not Verified":False, 'Source Verified':True, 'Verified':True})

    def __log_transform(self):
        variables = [
          "all_util", "annual_inc", "annual_inc_joint", "bc_open_to_buy", 
          "delinq_amnt", "dti", "max_bal_bc", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op",
          "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc", "mths_since_rcnt_il",
          "mths_since_recent_bc", "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl",
          "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts",
          "num_rev_tl_bal_gt_0", "num_sats", "open_acc", "open_acc_6m", "open_act_il",
          "open_il_12m", "open_il_24m", "open_rv_12m", "open_rv_24m", "pub_rec_bankruptcies",
          "revol_bal", "revol_bal_joint", "tax_liens", "tot_cur_bal", "tot_hi_cred_lim",
          "total_acc", "total_bal_ex_mort", "total_bal_il", "total_bc_limit", "total_cu_tl",
          "total_il_high_credit_limit", "total_rev_hi_lim"
        ]
        # 음수의 값을 가지는 행 삭제(로그 변환 불가 방지)
        self.df = self.df.loc[~(self.df[variables]<0).any(axis=1)]
        # 각 변수에 대해 로그 변환 수행 및 기존 변수 삭제
        for var in variables:
            # 로그 변환 후 변수 이름에 '_log'를 추가하여 새로운 변수 생성
            new_var = var + "_log"
            self.df[new_var] = np.log(self.df[var] + 1)  # 0이 아닌 값이어야 하므로 +1 추가
            # 기존 변수 삭제
            self.df.drop(columns=[var], inplace=True)
        return self.df
    
    def preprocess(self, is_train:bool=True)->None:
        # loan_status 제외 모든 column이 결측치(na)인 행 제거 (1개 행 제거됨)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        self.__preprocess_target_variable()
        # 결측치 제거
        self.__preprocessing_na(is_train)
        ## object 처리하기
        self.__preprocessing_objects()
        ## 다중공선성 제거 - 0419 추가
        self.__Multicollinearity()
        ## 로그변환 - 0419 추가
        self.__log_transform()
        # index 재설정
        self.df.reset_index(drop=True, inplace=True)
        if not is_train:
            self.df.dropna(subset=self.df.columns.difference(['loan_status']),inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        
    def do_scaling(self, scaler:StandardScaler|MinMaxScaler, fitted_scaler=None):
        all_features = self.df.columns
        numeric_features = [feature for feature in all_features if 'cat_' not in feature]
        
        print("numeric:", numeric_features)
        numeric_df = self.df[numeric_features].drop(columns=['loan_status', 'raw_total_pymnt'])
        if fitted_scaler==None:
            scaler = scaler()
            scaler.fit(numeric_df)
        else:
            scaler = fitted_scaler
        print("numeric shape", numeric_df.shape)
        numeric_features.remove('loan_status')
        numeric_features.remove('raw_total_pymnt')
        self.scaled_df = pd.DataFrame(data=scaler.transform(numeric_df), columns=numeric_features)
        return self.scaled_df, scaler
    
    def do_pca(self, threshold:float=0.95, fitted_pca=None, n_components=0):
        if fitted_pca==None:
            pca = PCA()
            pca.fit(self.scaled_df)
            explained_variance = pca.explained_variance_ratio_
            cumulative_explained_variance = np.cumsum(explained_variance)
            n_components = np.argmax(cumulative_explained_variance >= threshold) + 1
            column_names = [f'PC{i+1}' for i in range(n_components)]
            pca = PCA(n_components=n_components)
            pca.fit(self.scaled_df)
        else:
            pca = fitted_pca
            column_names = [f'PC{i+1}' for i in range(n_components)]
        pca_components = pca.transform(self.scaled_df).astype('float32')
        self.pca_df = pd.DataFrame(data=pca_components, columns=column_names)
        return pca, n_components
    
    def get_final_df(self, is_pca:bool):
        """PCA적용한데이터프레임"""
        all_features = self.df.columns
        print('all features', all_features)
        cat_features = [feature for feature in all_features if 'cat_' in feature]
        print('cat_features',cat_features)
        cat_df = self.df[cat_features].reset_index(drop=True)
        target_df = self.df[['loan_status']].reset_index(drop=True)
        if is_pca:
            pca_df = self.pca_df.reset_index(drop=True)
        else:
            numeric_features = [feature for feature in all_features if 'cat_' not in feature]
            print('numeric features', numeric_features)
            print(self.scaled_df.columns)
            pca_df = self.scaled_df
        self.df = pd.concat([cat_df, pca_df, target_df], axis=1)
        return self.df
    
    def get_df(self)->pd.DataFrame:
        return self.df

### 학습데이터(train set) 전처리

- ROE를 계산하기 위해서 필요한 전처리하지 않은 `total_pymnt` 컬럼을 `raw_total_pymnt` 컬럼으로 추가

In [3]:
origin_train = pd.read_csv("../data/lending_club_2020_train.csv")
origin_train['raw_total_pymnt'] = origin_train['total_pymnt']
origin_train.to_csv("lending_club_train.csv", index=False)

- 학습 데이터에 대해서 위에서 작성한 Preprocessor 클래스를 통해 전처리, scaling

In [9]:
train_preprocessor = Preprocessor()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
train_preprocessor.load_origin_file(file_path="lending_club_train.csv")
# drop_columns.txt 파일의 위치를 명시해주세요
train_preprocessor.drop_columns(drop_columns_file_path='drop_columns.txt')
# preprocess를 돌리면, addr_state를 제외한 object field 및, na(결측치) 처리됩니다.
train_preprocessor.preprocess(is_train=True)
train_df = train_preprocessor.get_df().copy(deep=True)
train_df.rename(columns={"loan_amnt":"raw_loan_amnt"}, inplace=True)
_, train_scaler = train_preprocessor.do_scaling(StandardScaler)
scaled_train_df = train_preprocessor.get_final_df(is_pca=False)
scaled_train_df = pd.concat([scaled_train_df, train_df[['raw_loan_amnt', 'raw_total_pymnt']]], axis=1)

numeric: ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'verification_status', 'loan_status', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_util', 'collections_12_mths_ex_med', 'application_type', 'dti_joint', 'acc_now_delinq', 'chargeoff_within_12_mths', 'mths_since_recent_inq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'raw_total_pymnt', 'is_after_2015', 'is_after_2012', 'fico_avg', 'all_util_log', 'annual_inc_log', 'annual_inc_joint_log', 'bc_open_to_buy_log', 'delinq_amnt_log', 'dti_log', 'max_bal_bc_log', 'mo_sin_old_il_acct_log', 'mo_sin_old_rev_tl_op_log', 'mo_sin_rcnt_rev_tl_op_log', 'mo_sin_rcnt_tl_log', 'mort_acc_log', 'mths_since_rcnt_il_log', 'mths_since_recent_bc_log', 'num_accts_ever_120_pd_log', 'num_actv_bc_tl_log', 'num_actv_rev_tl_log', 'num_bc_sats_log', 'num_bc_tl_log', 'num_il_tl_log', 'num_op_rev_tl_log', 'num_rev_accts_log', 'num_rev_tl_bal_gt_0_log', 'num_sats_log', 'open_a

- scaling이 끝난 데이터프레임 저장

In [12]:
scaled_train_df.to_csv("scaled_train.csv", index=False)

### 시험 데이터(test set) 전처리

- 위(train set)와 동일한 과정을 통해 scaled_test.csv를 생성한다

In [13]:
origin_test = pd.read_csv("../data/lending_club_2020_test.csv")
origin_test['raw_total_pymnt'] = origin_test['total_pymnt']
origin_test.to_csv("lending_club_test.csv", index=False)

In [14]:
test_preprocessor = Preprocessor()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
test_preprocessor.load_origin_file(file_path="lending_club_test.csv")
# drop_columns.txt 파일의 위치를 명시해주세요
test_preprocessor.drop_columns(drop_columns_file_path='drop_columns.txt')
# preprocess를 돌리면, addr_state를 제외한 object field 및, na(결측치) 처리됩니다.
test_preprocessor.preprocess(is_train=False)
test_df = test_preprocessor.get_df().copy(deep=True)
test_df['total_loan_amnt'] = test_df['loan_amnt']*(1+test_df['int_rate']/100)
test_df.rename(columns={"loan_amnt":"raw_loan_amnt"}, inplace=True)
test_preprocessor.do_scaling(scaler=StandardScaler, fitted_scaler=train_scaler)
scaled_test_df = test_preprocessor.get_final_df(is_pca=False)
scaled_test_df = pd.concat([scaled_test_df, test_df[['raw_loan_amnt', 'raw_total_pymnt']]], axis=1)


numeric: ['loan_amnt', 'term', 'int_rate', 'installment', 'sub_grade', 'emp_length', 'verification_status', 'loan_status', 'delinq_2yrs', 'inq_last_6mths', 'pub_rec', 'revol_util', 'collections_12_mths_ex_med', 'application_type', 'dti_joint', 'acc_now_delinq', 'chargeoff_within_12_mths', 'mths_since_recent_inq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'raw_total_pymnt', 'is_after_2015', 'is_after_2012', 'fico_avg', 'all_util_log', 'annual_inc_log', 'annual_inc_joint_log', 'bc_open_to_buy_log', 'delinq_amnt_log', 'dti_log', 'max_bal_bc_log', 'mo_sin_old_il_acct_log', 'mo_sin_old_rev_tl_op_log', 'mo_sin_rcnt_rev_tl_op_log', 'mo_sin_rcnt_tl_log', 'mort_acc_log', 'mths_since_rcnt_il_log', 'mths_since_recent_bc_log', 'num_accts_ever_120_pd_log', 'num_actv_bc_tl_log', 'num_actv_rev_tl_log', 'num_bc_sats_log', 'num_bc_tl_log', 'num_il_tl_log', 'num_op_rev_tl_log', 'num_rev_accts_log', 'num_rev_tl_bal_gt_0_log', 'num_sats_log', 'open_a

In [15]:
scaled_test_df.to_csv("scaled_test.csv", index=False)

- 전처리가 끝난 데이터의 차원이 맞는지 확인합니다.

In [11]:
train_df.shape, scaled_train_df.shape, test_df.shape, scaled_test_df.shape

((1131682, 88), (1131682, 89), (320678, 88), (320678, 89))